# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов, теорий заговоров, постов в помоечных телеграм каналах, текстов журналистов и СМИ с выразительным стилем). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
! pip install transformers

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
import torch

DEVICE = torch.device("cuda:0")

In [ ]:
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

В качестве данных для оубчения возьмем тексты песен исполнителя **Mujuice**

In [5]:
with open('mujuice_texts.txt', 'r') as file:
    text = file.read()

По сути, исходные тексты уже изначально похожи на сгенерированные...:

In [6]:
print(text[0:305])

Дизели
 Ветер
 По Настроению
 Юный Вечер Убит
 Значит Я Жив
 Сердца Бит Долбит Вслед
 Пегасы И Скутеры
 Гонки Со Встречными
 Дела Как Дела
 Hola!
 Пьяные Феи
 *ah-ah*
 Пахнут Сиреневым
 Рубят Ресницами
 Головы С Плеч
 Течь
 Легко Было Ключ
 Проглотить И Пролиться
 Раскаленными Каплями
 Cool-Cool Death!
 


In [ ]:
# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path='mujuice_texts.txt',block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

In [13]:
def generate_text(text, temperature=1, top_k=50, max_length=200):
  input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
  model.eval()
  with torch.no_grad():
      out = model.generate(input_ids, 
                          do_sample=True,
                          temperature = temperature,
                          top_k = top_k,
                          max_length = max_length
                          )
  generated_text = list(map(tokenizer.decode, out))[0]
  print()
  print(generated_text)

**temperature ≈ 0**

In [17]:
text = "Раз два три четыре пять "
generate_text(text, temperature=0.01, max_length=70)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Раз два три четыре пять 
 Два три четыре пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять пять
 Три четыре пять


**temperature = 1**

In [ ]:
text = "Раз два три четыре пять "
generate_text(text, temperature=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Раз два три четыре пять  Звуками войны
 Я вижу огни
 Укрепим сталью льда
 Я верю в чудеса
 Зовёт с собой
 Холодный Восток и Атланты
 Мне бы снова оказаться в Париже
 О Боже, что за ерунда
 В темноте звезда в ладонь
 Я слышал это в толпе
 Мы спели там, на пляже
 Потанцуй с огнём снова
 Я жду ответа
 Не найдётся, где взять ответ
 Жди, дождёмся, вернёмся
 Вместе из огня и молний
 Будем танцевать ещё
 Вместе на одной широте
 Звёзд не вернуть в прах
 Звезды смотрят на Восток
 Мне кажется, что мы одни
 Оу… мы одни на свете!
 Оу! мы одни! Ура ура! ура ура ура ура! ура ура! ура ура ура ура! ура ура ура ура ура! и мы летим!
 Потанцуй с огнём снова
 Давай на время забыть о льдах


**temperature = 0.75, top_k = 50**

In [93]:
generate_text("Раз два три четыре пять", temperature=0.75, max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Раз два три четыре пять
 Ищи меня по всей планете
 В каждой клеточке твоей
 Все на нуле...

 Три копейки
 Семь раз отмерь
 Жизнь одна
 Мы умрем
 Но сможем ли мы снова
 Сжаться в кучку
 Потерять контроль
 На секунду
 На два
 Раз два три четыре пять
 Ищи меня по всей планете
 В каждой клеточке твоей
 Все на нуле...

 Три копейки
 Семь раз отмерь
 Жизнь


In [25]:
generate_text("Небо, звезды и луна", temperature=0.75, max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Небо, звезды и луна
 Не знаю, что с тобой
 Я знаю, что с тобой
 Оу!

 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead
 I'm dead


**top_k=10**

In [123]:
generate_text("Небо, звезды и луна", temperature=0.75, max_length=50, top_k=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Небо, звезды и луна
 Я не знаю, где кончается лето
 Я не знаю, где начинается осень
 Я не знаю, где начинается весна
 Я не знаю, где кончается лето
 Я не знаю, где кончается осень
 Я не знаю


Настройка параметра num_beams не позволила получить какие-то разнообразные результаты:

**num_beams = 10**

In [152]:
input_ids = tokenizer.encode("День и ночь", return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=True, num_beams=10, temperature=0.75, max_length=40)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


День и ночь
 Оу!
 Оу!
 Оу!
 Оу!
 Оу!
 Оу!
 Оу!
 Оу!
 Оу!



**num_beams = 2**

In [172]:
input_ids = tokenizer.encode("День и ночь", return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=True, num_beams=2, temperature=0.75, max_length=40)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


День и ночь
 Cool-cool
 Cool-cool
 Cool-cool
 Cool-cool
 Cool-cool
 Cool-cool



### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

GPT-1: [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
<br>
GPT-2: [Language Models are Unsupervised Multitask Learners](https://d4mucfpksywv.cloudfront.net/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
<br>
GPT-3: [Language Models are Few-Shot Learnerslink text](https://arxiv.org/pdf/2005.14165.pdf)
<br><br>
2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

В качестве обучающих данных для модели GPT-3 использовался отфильтрованный корпус [Common Crawl](https://commoncrawl.org/). Датасет содержит петабайты данных в виде необработанных веб-страниц, сканированных с 2008 года. Разработчики GPT-3 взяли данные за период 2016-2019 годы. Помимо корпуса Common Crawl, из которого состояло 60% обучающей выборки, были также использованы датасеты WebText2, Books1, Books2 и Wikipedia.
<br><br>
Проблема корпуса Common Crawl, с которой столкнулись разработчики GPT-3, заключалась разном уровне качества текстов. Авторы решили проблему, отфильтровав корпус с помощью следующих техник:
1. Корпус CommonCrawl был пропущен через классификатор текстов по качеству. Чтобы обучить такой классификатор, в качестве позитивных примеров были взяты тексты из готовых отфильтрованных датасетов (Wikipedia, WebText, Books), в качестве отрицательных — неотфильтрованный датасет CommonCrawl.
2. Чтобы исключить из корпуса избытночные данные и предотвратить переобучение, разработчики применили дедупликацию данных (как и внутри каждого документа, так и на уровне всего корпуса).
3. Чтобы еще больше повысить разнообразие текстов, авторы расширили корпус CommonCrawl, добавив в обучающую выборку корпусы Wikipedia, WebText2 и Books.

Размер CommonCrawl 2016-2019 до фильтрации: 45 TB
<br>
После фильтрации: 570GB